#  Gradio 챗봇 구현 (간단한 QA 애플리케이션)

### **학습 목표:** LangChain의 LCEL을 활용하여 Gradio 기반의 AI 챗봇을 설계한다.

---

##  환경 설정

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Simple QA Chain  

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3, 
    )

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 체인 실행
response = chain.invoke({
    "user_input": "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?"
})

# AI의 응답 텍스트를 출력 
print(response)

파이썬에서 리스트를 정렬하는 방법은 여러 가지가 있습니다. 대표적인 방법 두 가지를 소개합니다.

1. **`list.sort()` 메서드**  
리스트 객체 자체를 정렬하며, 원본 리스트가 변경됩니다. 반환값은 `None`입니다.

```python
numbers = [3, 1, 4, 1, 5, 9]
numbers.sort()
print(numbers)  # 출력: [1, 1, 3, 4, 5, 9]
```

내림차순 정렬을 원하면 `reverse=True` 옵션을 사용합니다.

```python
numbers.sort(reverse=True)
print(numbers)  # 출력: [9, 5, 4, 3, 1, 1]
```

2. **`sorted()` 함수**  
원본 리스트는 변경하지 않고, 정렬된 새로운 리스트를 반환합니다.

```python
numbers = [3, 1, 4, 1, 5, 9]
sorted_numbers = sorted(numbers)
print(sorted_numbers)  # 출력: [1, 1, 3, 4, 5, 9]
print(numbers)         # 원본 리스트는 그대로: [3, 1, 4, 1, 5, 9]
```

역순 정렬도 가능합니다.

```python
sorted_numbers_desc = sorted(numbers, reverse=True)
print(sorted_numbers_desc)  # 출력: [9, 5, 4, 3, 1, 1]
```

---

추가로, 정렬 기준을 지정하고 싶을 때는 `key` 매개변수를 사용합니다.

예를 들어, 문자열 리스트를 길이순으로 정렬하기:

```python
words = ['apple', 'banana', 'cherry', 'date']
words.sort(key=len)
print(words)  # 출력: ['date', 'apple', 'banana', 'cherry']
```

필요하면 더 자세히 설명해 드릴 수 있습니다!


In [ ]:
# 마크다운 출력
from IPython.display import display, Markdown

display(Markdown(response))

## Gradio ChatInterface  
- 설치: pip install gradio --upgrade

### 1) 기본 구조

In [3]:
import gradio as gr

# 챗봇 함수 정의
def chat_function(message, history):
    return "응답 메시지"

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=chat_function,  # 실행할 함수
    analytics_enabled=False,  # 사용 정보 제공 여부
)

# 챗봇 인터페이스 실행
demo.launch()

/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [4]:
# 인터페이스 종료
demo.close()

Closing server running on port: 7860


### 2) 간단한 예제: Echo 챗봇

In [ ]:
def echo_bot(message, history):
    return f"당신이 입력한 메시지: {message}"

demo = gr.ChatInterface(
    fn=echo_bot,
    title="Echo 챗봇",
    description="입력한 메시지를 그대로 되돌려주는 챗봇입니다.",
    analytics_enabled=False,  
)

demo.launch()

In [ ]:
demo.close()

### 3) 스트리밍 응답

In [2]:
# 스트리밍 챗봇 함수 정의
import time

def streaming_bot(message, history):
    response = f"처리 중인 메시지: {message}"
    for i in range(len(response)):
        time.sleep(0.1)          # 0.1초 대기
        yield response[:i+1]

In [ ]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

In [ ]:
demo.close()

### 4) 추가 입력 컴포넌트
- 최대 응답 길이 등 기타 설정을 위한 추가 입력

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])


# 챗봇 함수 정의
def chat_function(message, history, model, temperature):

    if model == "gpt-4.1-mini":
        model = ChatOpenAI(model=model, temperature=temperature)
    elif model == "gemini-2.0-flash":
        model = ChatGoogleGenerativeAI(model=model, temperature=temperature)

    chain = prompt | model | StrOutputParser()

    response = chain.invoke({
        "user_input": message
    })
    return response

# 챗봇 인터페이스 생성
with gr.Blocks() as demo:
    model_selector = gr.Dropdown(["gpt-4.1-mini", "gemini-2.0-flash"], label="모델 선택")
    slider = gr.Slider(0.0, 1.0, label="Temperature", value=0.3, step=0.1, render=False)

    gr.ChatInterface(
        fn=chat_function, 
        additional_inputs=[model_selector, slider],
        analytics_enabled=False,  
    )

# 챗봇 인터페이스 실행
demo.launch()

In [ ]:
demo.close()

### 5) 예시 질문 설정

In [ ]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
    examples=[
        "파이썬 코드를 작성하는 방법을 알려주세요",
        "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?",
    ]    
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

In [ ]:
demo.close()    

### 6) 멀티모달 기능
- `multimodal=True` 옵션
- 이미지나 파일을 처리할 수 있는 멀티모달 챗봇 구현

- message 파라미터:
    ```python
    {
        "text": "user input", 
        "files": [
            "updated_file_1_path.ext",
            "updated_file_2_path.ext", 
            ...
        ]
    }
    ```
- history 파라미터:
    ```python
    [
        {"role": "user", "content": ("cat1.png")},
        {"role": "user", "content": ("cat2.png")},
        {"role": "user", "content": "What's the difference between these two images?"},
    ]
    ```

In [ ]:
import gradio as gr
import base64
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

def convert_to_url(image_path):
    """이미지를 URL 형식으로 변환"""
    with open(image_path, "rb") as image_file:
        # 이미지를 base64로 인코딩
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/jpeg;base64,{encoded_string}"

def multimodal_bot(message, history):

    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)
    
    if isinstance(message, dict):
        # 텍스트와 파일 추출
        text = message.get("text", "")
        
        # 히스토리와 현재 메시지에서 모든 파일 경로 추출
        filepath_list = []
        
        # 히스토리에서 이미지 파일 추출
        print("History:", history)  # 디버깅용
        for exchange in history:
            user_message = exchange[0]
            if isinstance(user_message, tuple):  # 이미지 메시지 확인
                filepath_list.append(user_message[0])
        
        # 현재 메시지의 파일들도 추가
        files = message.get("files", [])
        filepath_list.extend(files)
        
        print("Filepath list:", filepath_list)  # 디버깅용
        
        if filepath_list:
            # 모든 이미지 처리
            image_urls = []
            for file_path in filepath_list:
                try:
                    image_url = convert_to_url(file_path)
                    image_urls.append({"type": "image_url", "image_url": image_url})
                except Exception as e:
                    print(f"이미지 처리 중 오류 발생: {e}")
                    continue
            
            if not image_urls:
                return "이미지 처리 중 오류가 발생했습니다."
            
            # 메시지 구성
            content = [
                {"type": "text", "text": text if text else "이 이미지들에 대해 설명해주세요."},
                *image_urls
            ]
            
            try:
                # API 호출
                response = model.invoke([
                    HumanMessage(content=content)
                ])
                return response.content
            except Exception as e:
                return f"모델 응답 생성 중 오류가 발생했습니다: {str(e)}"
        
        return text if text else "이미지를 업로드해주세요."
    
    return "텍스트나 이미지를 입력해주세요."

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=multimodal_bot,
    multimodal=True,
    title="멀티모달 챗봇",
    description="텍스트와 이미지를 함께 처리할 수 있는 챗봇입니다. 이전 대화의 이미지들도 함께 고려합니다.",
    analytics_enabled=False,  
    textbox=gr.MultimodalTextbox(placeholder="텍스트를 입력하거나 이미지를 업로드해주세요.", file_count="multiple", file_types=["image"]),
)

# 인터페이스 실행
demo.launch()

In [ ]:
demo.close()

### 7) PDF 뷰어
- 설치: pip install gradio_pdf 또는 poetry add gradio_pdf

In [ ]:
from gradio_pdf import PDF

def answer_invoke(message, history):   
    return message

with gr.Blocks(
    analytics_enabled=False,  
) as demo:
    with gr.Row():
        # API Key Section
        api_key_input = gr.Textbox(
            label="Enter OpenAI API Key",
            type="password",
            placeholder="sk-..."
        )
        
    with gr.Row():
        # PDF Upload and Chat Interface
        with gr.Column(scale=2):
            pdf_file = PDF(
                label="Upload PDF File",
                height=600,  # PDF 뷰어 높이 설정
            )
        with gr.Column(scale=1):
            chatbot = gr.ChatInterface(
                fn=answer_invoke,
                title="PDF-based Chatbot",
                description="Upload a PDF file and ask questions about its contents.",
            )


demo.launch()

In [ ]:
demo.close()

## Memory 추가

In [7]:
# chat_history 플레이스홀더를 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("system", "이전 대화 내용을 참고하여 질문에 대해서 친절하게 답변합니다."),
    ("human", "{user_input}")
])

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):
    
    print(history)

    history_messages = []
    # for msg in history:
    #     if msg['role'] == "user":
    #         history_messages.append(HumanMessage(content=msg['content']))
    #     elif msg['role'] == "assistant":
    #         history_messages.append(AIMessage(content=msg['content']))

    
    for pair in history:
        if isinstance(pair, list) and len(pair) == 2:
            user_msg, ai_msg = pair
            history_messages.append(HumanMessage(content=user_msg))
            history_messages.append(AIMessage(content=ai_msg))

    history_messages.append(HumanMessage(content=message))
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response
    

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="파이썬 코드 어시스턴트", # 채팅 인터페이스의 제목
    )

# Gradio 인터페이스 실행
demo.launch()

/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [8]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7862


# [실습 프로젝트]

- **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

In [13]:
# chat_history 플레이스홀더를 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


PLANMANGER_SYSTEM_ROLE = "당신은 여행 플래너입니다. 한국어로만 응답하고, 톤은 밝고 부드러운 세일즈 톤을 유지합니다."

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7,
    top_p=0.9,
    presence_penalty=0.8, # NOTE: 몇 퍼센트가 적당할지 테스트 해봐야할듯
    frequency_penalty = 0.6
)

# 기본 출력 파서
output_parser = StrOutputParser()

# 1. 사용자 입력 분석
analyze_user_request = ChatPromptTemplate.from_messages([
    ("system", PLANMANGER_SYSTEM_ROLE),
    MessagesPlaceholder("chat_history"),
    ("system", """
    사용자가 입력한 요청사항과 이전 대화 기록을 바탕으로 여행 계획에 필요한 기본 정보를 분석해줘

    [사용자 요청사항]
    {user_input}

    [분석할 기본 정보와 요청사항]
    
    분석한 기본정보
    - 여행자 인원 구성
      - 여행자들의 특성, 나이대, 여행자들 관계 등
    - 여행 목적
    - 여행 예산 범위
    - 여행 성향(액티비티를 좋아하는지, 휴양을 좋아하는지, 활동 선호도 등)
    - 여행 기간
    - 여행지 특성

    요청사항 예시
    1. 위 정보를 바탕으로 일자별 여행 계획을 4박 5일 일정으로 구성해줘.
    2. 무리한 이동은 피하고, 하루에 2~3개 정도의 방문지를 배치해줘.
    3. 아침/점심/저녁 식사도 추천해줘 (가능하면 지역 맛집 포함).
    4. 각 일정의 설명에 간단한 이유도 함께 적어줘.

    [출력 형식은 다음과 같이 json으로 나오게 해줘]
    {text : "{분석한 내용}"}
    """)
])

analyze_user_request_chain = analyze_user_request | model | JsonOutputParser()

# 2. 일정 뼈대 생성
plan_prompt = ChatPromptTemplate.from_template("""
너는 전문 여행 플래너야. 아래 고객 정보를 참고하여 일자별 기본 여행 일정을 구성해줘
여행자 특성에 맞는 일정들로 추천해서 넣어줘
                                                    
[요청한 기본 정보]
{text}

일정 구성 형식 예시

Day 1 - 도착 & 여유로운 시작  
- [ ] 공항 도착 및 렌터카 수령 (렌터카로 이동이 편리한 지역 특성 고려)  
- [ ] 숙소 체크인 (아이와 함께 쉬기 좋은 숙소)  
- [ ] 근처 해변 산책 (비행 피로를 풀기 위한 가벼운 일정)  
- 🍴 점심: 공항 근처 고기국수 맛집  
- 🍴 저녁: 숙소 근처 흑돼지 전문점

Day 2 - 자연 체험 & 가벼운 산책  
- [ ] 용머리해안 방문 (아이와 함께 걷기 좋은 해안 절경)  
- [ ] 카멜리아힐 산책 (꽃과 자연을 즐기기 좋은 명소)  
- [ ] 숙소 근처 카페에서 휴식  
- 🍴 아침: 숙소 조식  
- 🍴 점심: 마라도 해물라면  
- 🍴 저녁: 협재 해물 뚝배기

...

이런 형식으로 일정을 구성해줘.
                                                    
[출력 형식은 다음과 같이 json으로 나오게 해줘]
{text : "{일정 구성 내용}"}
""")
plan_prompt_chain = plan_prompt | model | JsonOutputParser()

# 3. 일정을 바탕으로 세부 일정을 계획하는 프롬프트
detail_plan_prompt = ChatPromptTemplate.from_template("""
아래 일정 구성 내용을 바탕으로 세부 일정을 완성해줘

일정 구성 
{text}

출력 형식은 전체 일정 계획표로 만들어줘
""")

detail_plan_chain = detail_plan_prompt | model | StrOutputParser()

# 4. 병렬 처리 체인 구성
chain = analyze_user_request_chain | plan_prompt_chain | detail_plan_chain


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):
    
    history_messages = []
    
    for pair in history:
        if isinstance(pair, list) and len(pair) == 2:
            user_msg, ai_msg = pair
            history_messages.append(HumanMessage(content=user_msg))
            history_messages.append(AIMessage(content=ai_msg))

    history_messages.append(HumanMessage(content=message))
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response
    
initial_message = [
    ("""안녕하세요! 안전하고 즐거운 여행계획을 추천해드리겠습니다."""),
    ("""어떤 곳으로 여행하고 싶으신가요?""")
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    chatbot=gr.Chatbot(value=initial_message),
    title="파이썬 코드 어시스턴트",
)

# Gradio 인터페이스 실행
demo.launch()

/var/folders/mh/x4qd1c7902q_114__5tjbjmw0000gn/T/ipykernel_75438/2019248964.py:131: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(value=initial_message),
/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import List

# 시스템 프롬프트
PLANMANGER_SYSTEM_ROLE = "당신은 여행 플래너입니다. 한국어로만 응답하고, 톤은 밝고 부드러운 세일즈 톤을 유지합니다."

# 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7,
    top_p=0.9,
    presence_penalty=0.8,
    frequency_penalty=0.6
)

# 파서 정의
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

# 사용자 요청 분석 프롬프트
analyze_user_prompt = ChatPromptTemplate.from_messages([
    ("system", PLANMANGER_SYSTEM_ROLE),
    MessagesPlaceholder("chat_history"),
    ("system", """다음 사용자 입력과 이전 대화 기록을 참고하여 여행 계획에 필요한 정보를 요약해줘:

[사용자 요청]
{user_input}
     
[분석할 기본 정보와 요청사항]

분석한 기본정보
- 여행자 인원 구성
    - 여행자들의 특성, 나이대, 여행자들 관계 등
- 여행 목적
- 여행 예산 범위
- 여행 성향(액티비티를 좋아하는지, 휴양을 좋아하는지, 활동 선호도 등)
- 여행 기간
- 여행지 특성

요청사항 예시
1. 위 정보를 바탕으로 일자별 여행 계획을 4박 5일 일정으로 구성해줘.
2. 무리한 이동은 피하고, 하루에 2~3개 정도의 방문지를 배치해줘.
3. 아침/점심/저녁 식사도 추천해줘 (가능하면 지역 맛집 포함).
4. 각 일정의 설명에 간단한 이유도 함께 적어줘.

출력 형식:
{{"text": "요약된 여행 정보"}}
""")
])
analyze_user_chain = analyze_user_prompt | model | json_parser

# 일정 구성 프롬프트
plan_prompt = ChatPromptTemplate.from_template("""
아래 고객 정보를 참고하여 일자별 기본 여행 일정을 구성해줘:

[고객 정보]
{text}

일정 구성 형식 예시

Day 1 - 도착 & 여유로운 시작  
- [ ] 공항 도착 및 렌터카 수령 (렌터카로 이동이 편리한 지역 특성 고려)  
- [ ] 숙소 체크인 (아이와 함께 쉬기 좋은 숙소)  
- [ ] 근처 해변 산책 (비행 피로를 풀기 위한 가벼운 일정)  
- 🍴 점심: 공항 근처 고기국수 맛집  
- 🍴 저녁: 숙소 근처 흑돼지 전문점

Day 2 - 자연 체험 & 가벼운 산책  
- [ ] 용머리해안 방문 (아이와 함께 걷기 좋은 해안 절경)  
- [ ] 카멜리아힐 산책 (꽃과 자연을 즐기기 좋은 명소)  
- [ ] 숙소 근처 카페에서 휴식  
- 🍴 아침: 숙소 조식  
- 🍴 점심: 마라도 해물라면  
- 🍴 저녁: 협재 해물 뚝배기
                            

출력 형식:
{{"text": "일정 구성 내용"}}

""")
plan_chain = plan_prompt | model | json_parser

# 상세 구성 프롬프트
detail_prompt = ChatPromptTemplate.from_template("""
아래 일정을 참고해서 각 일정을 더 구체화해줘:

{text}
                                                 
[출력할 내용은 아래와 같아]
날짜, 시간대, 일정/장소, 설명
""")
detail_chain = detail_prompt | model | str_parser


def convertParser(output):
    return {"text": output}

# 전체 체인 조립
chain = (
    analyze_user_chain
    | convertParser
    | plan_chain
    | convertParser
    | detail_chain
)


# 상세 구성 프롬프트
basic_prompt = ChatPromptTemplate.from_messages([
    ("system", PLANMANGER_SYSTEM_ROLE),
    MessagesPlaceholder("chat_history"),
    ("system", """다음 사용자 입력과 이전 대화 기록을 참고해서 아직 물어보지않은 기본 정보를 물어봐줘
     지금 물어볼 질문을 마지막으로 더 물어볼 질문이 없는 경우, 마지막 질문일 경우 마지막이라는걸 알려주고, 이 정보를 바탕으로 계획을 짤거라고 안내해줘

[물어볼 기본 정보 리스트]
{answer_list}
     
[사용자 요청]
{user_input}
""")
])
basic_chain = basic_prompt | model | str_parser

# Gradio 함수
def answer_invoke(message, history):        
    return invoke_chain(message, previous_history_to_pair(history).append(HumanMessage(content=message)))

# NOTE: 이전 채팅 기록을 pair 리스트로 전처리합니다.
def previous_history_to_pair(history) -> List[tuple[str, str]] :
    history_messages = []
    for pair in history:
        if isinstance(pair, list) and len(pair) == 2:
            history_messages.append(HumanMessage(content=pair[0]))
            history_messages.append(AIMessage(content=pair[1]))

    return history_messages

# NOTE: 사용자와 대화할 chain을 선택합니다.
def invoke_chain(message, history_messages):
    answer_list = [
        "여행자 인원 구성은 어떻게 되나요?",
        "여행자들은 어떤 특성과 나이대를 가지고 있으며, 서로 어떤 관계인가요?",
        "이번 여행의 주된 목적은 무엇인가요?",
        "여행 예산은 어느 정도로 계획하고 계신가요?",
        "여행 중 어떤 성향을 선호하시나요? (예: 액티비티, 휴양, 관광 등)",
        "여행 기간은 언제부터 언제까지인가요?",
        "여행지에 대해 선호하는 특성이 있나요? (예: 자연 중심, 도시 중심, 렌터카 필요 여부 등)"
    ]
    
    if (len(history_messages) >= len(answer_list)):
        return chain.invoke({
            "chat_history": history_messages,
            "user_input": message
        })
    else: 
        return basic_chain.invoke({
            "chat_history": history_messages,
            "answer_list": answer_list,
            "user_input": message
        })

# 초기 메시지
initial_message = [
    ("여행 플래너 요청", "안녕하세요! 어떤 곳으로 여행하고 싶으신가요?")
]

# 인터페이스 설정
demo = gr.ChatInterface(
    fn=answer_invoke,
    chatbot=gr.Chatbot(value=initial_message),
    title="여행 플래너 어시스턴트"
)

demo.launch()

/var/folders/mh/x4qd1c7902q_114__5tjbjmw0000gn/T/ipykernel_75438/619383134.py:172: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(value=initial_message),
/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import List
from langchain_core.runnables import RunnableParallel


def get_chain():
    summarize_templete = """
    {text}

    위에 입력된 텍스트를 다음 항목으로 요약해주세요: 
    - 여행 일정 :
    - 교통편 일정 :
    - 여행 장소 :
    - 여행 스타일 :
    - 예산 :
    - 추천 숙소 :"""

    summarize_prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 여행 일정 작성을 도와주는 AI 어시스턴트입니다."),
        MessagesPlaceholder("chat_history"),
        ("human", summarize_templete)
    ])

    planner_prompt = ChatPromptTemplate.from_template("""
    다음 텍스트의 여행 일정을 기반으로 세부 여행 일정을 짜주세요.
    텍스트: {summary}
    규칙:
    1. 날짜 및 시간과 장소, 세부 계획 항목으로 표 형태로 작성하세요.
    2. 여행 스타일과 추천 숙소, 예산에 맞추어 동선을 고려하여 장소를 추천하세요.
    답변:""")

    # 문자열 출력 파서
    output_parser = StrOutputParser()

    # 체인 구성
    model = ChatOpenAI(
        model="gpt-4.1",
        temperature=0.4,
        top_p=0.7
    )

    # 요약 체인
    summarize_chain = summarize_prompt | model 

    # 감정 분석 체인
    planner_chain = planner_prompt | model | output_parser

    # 전체 체인
    chain = (
        summarize_chain 
        | RunnableParallel(
            summary=lambda x: x.content,
            plan=lambda x: planner_chain.invoke({"summary": x.content}),
        )
    )
    return chain


def answer_invoke(message, history):
    chain = get_chain()
    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    response = chain.invoke({
        "chat_history": history_messages,
        "text": message
    })
    response = f"<요약>\n{response['summary']}\n\n<일정>\n{response['plan']}"
    return response

demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="여행 일정 어시스턴트", # 채팅 인터페이스의 제목
    type="messages"
)

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2147, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harryi/modu_llm3/faq_bot-1/.venv/lib/python3.12/site-packages/gradio/utils.py", l